<a href="https://colab.research.google.com/github/wisesprout/newspaper_to_online_newsletter/blob/main/Nextlab_news_category_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-_29u7eqz
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-_29u7eqz
     |████████████████████████████████| 132 kB 4.3 MB/s 
     |████████████████████████████████| 344 kB 34.9 MB/s 
     |████████████████████████████████| 47.3 MB 1.8 MB/s 
     |████████████████████████████████| 4.5 MB 48.2 MB/s 
     |████████████████████████████████| 1.2 MB 49.3 MB/s 
     |████████████████████████████████| 3.8 MB 57.2 MB/s 
     |████████████████████████████████| 6.5 MB 70.8 MB/s 
     |████████████████████████████████| 67 kB 5.4 MB/s 
     |████████████████████████████████| 596 kB 68.1 MB/s 
     |████████████████████████████████| 895 kB 63.1 MB/s 
     |████████████████████████████████| 8.6 MB 38.1 MB/s 
     |████████████████████████████████| 79 kB 9.5 MB/s 
     |████████████████████████████████| 127 kB 63.3 MB/s 
  Created wheel for kobert: filename=kob

In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, notebook

import os


from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [4]:
# 사용할 device 설정
device = torch.device('cpu')

# get pretrained bert and vocab
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


#Load Data

In [5]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


- news_class9x1400 = 9개 카테고리별 1400개 기사
- 1개 기사 = 줄 바꿈으로 대부분의 문장 구분


In [35]:
dataset_train = []
dataset_test = []

root = "/content/drive/MyDrive/Colab Notebooks/CP2/news_class9x1400/"
list = os.listdir(root)

# delete '.DS_Store'
list.pop(1)

# create dictionary for news categories
# {0: 'sport', 1: 'culture', 2: 'politic', 3: 'social', 4: 'life', 5: 'health', 6: 'economy', 7: 'entertainment', 8: 'ITscience'}
cat_dict = {}
for cat in list:
    cat_dict[list.index(cat)] = cat

for cat in list:   # 파일을 읽어서 기사 내용 + 카테고리를 하나의 list 데이터로 생성
    files = os.listdir(root + cat)  # n번째 카테고리의 파일 경로
    for i,f in enumerate(files):  # i=index, f=i번째 기사 파일명
        fname = root + cat + "/" + f  # i번째 기사 파일 경로
        file = open(fname, "r")  # 'r'=read
        strings = file.read()

        cat_num = str(list.index(cat))  # 카테고리의 인덱스

        # train:test = 17:3 = 1190개:210개
        if i < 1190:
            dataset_train.append([strings, cat_num])  # 카테고리 이름 대신 인덱스를 str으로 대입
        else:
            dataset_test.append([strings, cat_num])  # 카테고리 이름 대신 인덱스 str으로 대입
        file.close()

# train set = 1190개 기사 * 9개 카테고리 = 10710, 
# test set = 210개 기사 * 9개 카테고리 = 1890
print(len(dataset_train), len(dataset_test))  

10710 1890


In [37]:
# first data of train set
# 뉴스데이터, 카테고리 번호
dataset_train[0]

['경사진 잔차길, 넌 너무 매력 없어\n우와~! 여기 참말로 잘해놨네.\n그래 억수로 잘해 놨다.\n자전거 길이 이만큼만 되어도 다닐만한데.\n가만 그런데 여기서는 어떻게 가야 하지?\n새로 깔린 자전거 길이 그만 찻길을 만나 끊어졌어요.\n잠깐 멈춰서 이쪽저쪽 번갈아가며 차가 오는지 살펴봅니다.\n마땅히 건널목도 따로 없고 찻길을 가로질러 가야 하기에 조심해야 했어요.\n그러나 생각보다 차가 그다지 많지 않은 길이라서 다행이었지요.\n아아~ 아까 저 아래에 개천 옆으로 빠져나가던 길이 있더니, 거기로 가면 찻길을 건너지 않아도 갈 수 있겠구나.\n엥? 그랬어?\n아아! 거기! 그렇구나.\n그럼 자전거 길로 가다가 그 밑으로 빠져나가면 이 길 건너편으로 나올 수 있는 거구나.\n그래. 그렇게 가면 되겠네.\n그래도 자전거 길 만들면서 꽤 생각하고 만들었네.\n모든 자전거 길이 이만큼만 된다면 잔차 타고 다니는 데 불편한 게 없겠다.\n우리 아까 거기 다시 가보자.\n실제로 길을 어떻게 만들어놨는지 한 번 타봤으면 좋겠는데?\n까짓 거 그러지\n뭐, 어차피 오늘은 잔차 길 보러 나온 거니까….\n동락공원을 지나 개천(이계천) 옆으로 난 자전거 길을 따라 구미산업단지 3공단, LG전자 TV모니터 공장과 삼성코닝정밀유리 회사 앞까지 내달립니다.\n얼마 전에 새로 깔아놓은 곳인데, 매우 깔끔하고 생각보다 설계를 아주 잘했더군요.\n어디에서든지 자전거 길을 따라오다가 찻길을 만나면 위험을 감수하고 건너가야 하는 불편함이 많이 있었는데, 이곳은 찻길을 따로 건너지 않고도 너끈히 갈 수 있도록 길을 잘 이어놓았더군요.\n모든 자전거 길이 이만큼만 된다면, 여러 가지 불편함 없이 잘 다닐 수 있을 듯했답니다.\n자전거=삶인 나, 자전거길 돌아봤더니\n자전거를 타고 일터에도 가고, 나들이도 가고, 그야말로 자전거가 삶인 사람이 바로 저랍니다.\n그렇기에 자전거 정책이나 자전거 길을 보는 눈이나 마음이 남다른 건 어쩔 수 없는 일이겠지요?\n틈틈이 기사나 엄지뉴스로도 내가 자

In [38]:
# tokenize vocab
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [39]:
# dataset class for training
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, 
                                                   max_seq_length=max_len, 
                                                   pad=pad, 
                                                   pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [40]:
# setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [41]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5, shuffle=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


#Make Model

In [42]:
# model for classification
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=9,  # for 9 categories
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device), return_dict=False)

        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out) 

In [43]:
# necessities for training
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()


t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [44]:
# accuracy while training
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

#Train Model

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(notebook.tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        train_acc += calc_accuracy(out, label)
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(notebook.tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/168 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 train acc 0.46628111471861466


  0%|          | 0/30 [00:00<?, ?it/s]

epoch 1 test acc 0.7656556372549019


...

  0%|          | 0/168 [00:00<?, ?it/s]

epoch 3 train acc 0.8816203327922078


  0%|          | 0/30 [00:00<?, ?it/s]

epoch 3 test acc 0.7939338235294117


  0%|          | 0/168 [00:00<?, ?it/s]

#Test

In [ ]:
# function for testing any input text
def softmax(vals, idx):
    valscpu = vals.cpu().detach().squeeze(0)
    a = 0
    for i in valscpu:
        a += np.exp(i)
    return ((np.exp(valscpu[idx]))/a).item() * 100

torch.save(model.state_dict(), "news.pt")
modelload = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
modelload.load_state_dict(torch.load("news.pt", device))

def testModel(model, seq):
    cate = ["정치","경제","사회", "생활/문화","세계","기술/IT", "연예", "스포츠"]
    tmp = [seq]
    transform = nlp.data.BERTSentenceTransform(tok, max_len, pad=True, pair=False)
    tokenized = transform(tmp)

    modelload.eval()
    result = model(torch.tensor([tokenized[0]]).to(device), [tokenized[1]], torch.tensor(tokenized[2]).to(device))
    idx = result.argmax().cpu().item()
    print("뉴스의 카테고리는:", cate[idx])
    print("신뢰도는:", "{:.2f}%".format(softmax(result,idx)))

In [ ]:
# testing
testModel(model, "신형 아이패드 프로에 m1칩 탑재 예정")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)


뉴스의 카테고리는: 기술/IT
신뢰도는: 96.87%
